In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<h1 style='color:white; background:blue; border:0'><center>Show US the Data: Start Our Study</center></h1>

![](https://oerc.osu.edu/sites/oerc/themes/oerc/images/projects/coleridge.png)

This competition challenges data scientists to show how publicly funded data and evidence are used to serve science and society. Data, evidence, and science are critical if government is address the many threats facing society: pandemics, climate change and coastal inundation, Alzheimer’s disease, child hunger, and support science and innovation, increase food production, maintain biodiversity, and address many other challenges. Yet much of the information about data necessary to inform evidence and science is locked inside publications.

Can natural language processing find the hidden-in-plain-sight data citations? Can machine learning find the link between the words used in research articles and the data referenced in the article?

### Coleridge Initiative - String Literals¶
The is a fairly naive approach to solving this problem.

* loop over train.csv
    * perform basic string cleaning (lowercase + remove non-alphanumeric)
    * create a lookup table for each possible description string (pub_title, dataset_title, dataset_label)
    * map it back to the expected cleaned_label string
* brute force search the test dataset for any string literals found in train.csv
    * if multiple matches are found, then pick the one with the most matches

In [ ]:
import simplejson
import re
import pydash
import sys
import os
from collections import defaultdict
from typing import *
from joblib import Parallel, delayed
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

import tqdm
from tqdm.auto import tqdm as tqdmp
tqdmp.pandas()

# NLP
import unicodedata, string, re, nltk, json
from nltk.corpus import stopwords
from nltk.util import ngrams
from wordcloud import WordCloud, STOPWORDS
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.dates as mdates

# ignoring warnings
import warnings
warnings.simplefilter("ignore")

# Some settings for visualizations
sns.set(rc={'axes.facecolor':'black', 'figure.facecolor':'black', 
            'xtick.color': 'white', 'ytick.color': 'white', 
            'grid.color': 'white', 'axes.labelcolor': 'white',
            'figure.dpi': 150, 'grid.linestyle': ':', 'grid.alpha': .6,
            'font.family': 'fantasy'})

In [ ]:
train = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')
ss = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/sample_submission.csv')
train.head()

In [ ]:
ss.head()

### EDA

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
# Note that ALL ground truth texts have been cleaned for matching purposes using the following code:
def text_cleaner(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()

def show_wordcloud(data, stop, mask = None, title = None, color = 'black'):
    """
    Function for creating wordclouds (with or without mask)
    """
    from wordcloud import WordCloud, ImageColorGenerator
    wordcloud = WordCloud(background_color = color,
                         stopwords = stop,
                         mask = mask,
                         max_words = 100,
                         scale = 3,
                         width = 4000, 
                         height = 2000,
                         collocations = False,
                         colormap = 'rainbow',
                         random_state = 1)
    
    wordcloud = wordcloud.generate(data)
    
    plt.figure(1, figsize = (16, 8), dpi = 300)
    plt.title(title, size = 15)
    plt.axis('off')
    if mask is None:
        plt.imshow(wordcloud, interpolation = "bilinear")
        plt.show()
    else:
        image_colors = ImageColorGenerator(mask)
        plt.imshow(wordcloud.recolor(color_func = image_colors), 
                   interpolation = "bilinear")
        plt.show()

##### Let's at first analyze the train pub_title data.

In [ ]:
title_cleaned = train['pub_title'].apply(lambda x: text_cleaner(x))

In [ ]:
title_length = title_cleaned.str.len()

plt.title('Title length', size = 15, color = 'white')
sns.distplot(title_length, kde = False, color = 'blue', 
             hist_kws = dict(alpha = 1))
plt.xlabel('Length of title (symbols)')
plt.show()

In [ ]:
title_words = title_cleaned.str.split().map(lambda x: len(x))

plt.title('Title words', size = 15, color = 'white')
sns.distplot(title_words, kde = False, color = 'blue', 
             hist_kws = dict(alpha = 1))
plt.xlabel('Length of title (words)')
plt.show()

In [ ]:
title_word_len = title_cleaned.str.split().apply(lambda x: [len(i) for i in x]).map(lambda x: np.mean(x))

plt.title('Title words length', size = 15, color = 'white')
sns.distplot(title_word_len, kde = False, color = 'blue', 
             hist_kws = dict(alpha = 1))
plt.xlabel('Mean word length in title (symbols)')
plt.show()

In [ ]:
words = title_cleaned.str.split().values.tolist()
title_corpus = [word for i in words for word in i]

title_counter = Counter(title_corpus)
title_most = title_counter.most_common()

stop = set(stopwords.words('english'))

title_top_words, title_top_words_count = [], []
for word, count in title_most[:100]:
    if word not in stop:
        title_top_words.append(word)
        title_top_words_count.append(count)

In [ ]:
plt.title('TOP-10 title words', color = 'white', size = 15)
sns.barplot(y = title_top_words[:10], x = title_top_words_count[:10], 
            edgecolor = 'black', color = 'blue')
plt.show()

In [ ]:
title_word_string = ' '.join(title_corpus)
show_wordcloud(title_word_string, stop)

## Extract all publication texts

Publications are provided in JSON format, broken up into sections with section titles. Let's extract all the texts with a little function.

In [ ]:
def text_extractor(url_id):
    url = '../input/coleridgeinitiative-show-us-the-data/train/{}.json'.format(url_id)
    return ' '.join(pd.read_json(url).text)

Note that there are multiple rows for some training documents, indicating multiple mentioned datasets

In [ ]:
train_texts = train.drop_duplicates(subset = ['Id'])['Id'].progress_apply(lambda x: text_extractor(x))

In [ ]:
train_texts

In [ ]:
text_cleaned = train_texts.progress_apply(lambda x: text_cleaner(x))

In [ ]:
text_cleaned

In [ ]:
text_length = text_cleaned.str.len()

plt.title('Pub text length', size = 15, color = 'white')
sns.distplot(text_length, kde = False, color = 'blue', 
             hist_kws = dict(alpha = 1))
plt.xlabel('Length of pub text (symbols)')
plt.show()

In [ ]:
text_words = text_cleaned.str.split().map(lambda x: len(x))

plt.title('Pub text words', size = 15, color = 'white')
sns.distplot(text_words, kde = False, color = 'blue', 
             hist_kws = dict(alpha = 1))
plt.xlabel('Length of pub text (words)')
plt.show()

In [ ]:
text_word_len = text_cleaned.str.split().progress_apply(lambda x: [len(i) for i in x]).map(lambda x: np.mean(x))

plt.title('Pub text words length', size = 15, color = 'white')
sns.distplot(text_word_len, kde = False, color = 'blue', 
             hist_kws = dict(alpha = 1))
plt.xlabel('Mean word length in pub text (symbols)')
plt.show()

In [ ]:
words = text_cleaned.str.split().values.tolist()
text_corpus = [word for i in words for word in i]

text_counter = Counter(title_corpus)
text_most = title_counter.most_common()

stop = set(stopwords.words('english'))

text_top_words, text_top_words_count = [], []
for word, count in text_most[:100]:
    if word not in stop:
        text_top_words.append(word)
        text_top_words_count.append(count)

In [ ]:
plt.title('TOP-20 text words', color = 'white', size = 15)
sns.barplot(y = text_top_words[:20], x = text_top_words_count[:20], 
            edgecolor = 'black', color = 'blue')
plt.show()

The goal in this competition is not just to match known dataset strings but to generalize to datasets that have never been seen before using NLP and statistical techniques. Not all datasets have been identified in train, but you have been provided enough information to generalize.

The objective of the competition is to identify the mention of datasets within scientific publications. Your predictions will be short excerpts from the publications that appear to note a dataset.

Submissions are evaluated on a Jaccard-based FBeta score between predicted texts and ground truth texts, with Beta = 0 (an F0 or precision score). Multiple predictions are delineated with a pipe (|) character in the submission file.

The following is Python reference code for the Jaccard score:

In [ ]:
train_df = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')
train_df

### Extract String Literals
Lets create a lookup table for all possible strings used to describe each dataset

In [ ]:
def clean_text(text: str) -> str:               return re.sub('[^A-Za-z0-9]+', ' ', str(text).lower()).strip()
def clean_texts(texts: List[str]) -> List[str]: return [ clean_text(text) for text in texts ] 

In [ ]:
def generate_lookup(df):
    lookup = defaultdict(set)
    for _, row in df.iterrows():
        lookup[ row['cleaned_label'] ] |= set(clean_texts([ row['dataset_label'], row['dataset_title'], row['pub_title'] ]))
    return lookup

next(iter(generate_lookup(train_df).items()))

### Train Dataset Validation
This validates that this algoritm works on the training dataset, and produces a 100% score

In [ ]:
def read_json(index: str, test_train="test") -> Dict:
    filename = f"../input/coleridgeinitiative-show-us-the-data/{test_train}/{index}.json"
    with open(filename) as f:
        json = simplejson.load(f)
    return json
        
def json2text(index: str, test_train="test") -> str:
    json  = read_json(index, test_train)
    texts = [
        row["section_title"] + " " + row["text"] 
        for row in json
    ]
    texts = clean_texts(texts)
    text  = " ".join(texts)
    return text


def extract_label(text: str, lookup: Dict[str, Set[str]]) -> str:
    labels = []
    for label, values in lookup.items():
        if any([
            value in text
            for value in values
        ]):
            labels += [ label.strip() ]
            
    # label = "|".join(set(labels))  # multi label support
    label = Counter(labels).most_common(1)[0][0] if len(labels) else ""  # single most-popular label
    # print('extract_label', labels, '->', label)
    return label

In [ ]:
%%time
def train_accuracy(df, limit=sys.maxsize) -> float:
    limit   = 100 if os.environ.get('KAGGLE_KERNEL_RUN_TYPE', 'Localhost') == 'Interactive' else limit
    lookup  = generate_lookup(df)
    labels  = Parallel(-1)(
        delayed(extract_label)(json2text(index, "train"), lookup)
        for index in df['Id'][:limit]
    )
    correct   = 0
    expecteds = df['cleaned_label'][:limit]
    for label, expected in zip(labels, expecteds):
        expected_set = set(expected.split("|"))
        label_set    = set(label.split("|"))
        matches      = expected_set & label_set
        correct     += len(matches) / len(label_set)

    # correct = np.count_nonzero( np.array(labels) == expecteds )
    total   = len(expecteds)
    return correct / total

train_accuracy(train_df, 100)

### Submission

In [ ]:
def generate_submission():
    submission_df = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/sample_submission.csv', index_col=0)
    lookup  = generate_lookup(train_df)
    indexes = submission_df.index
    labels  = Parallel(-1)(
        delayed(extract_label)(json2text(index, "test"), lookup)
        for index in indexes
    )
    submission_df['PredictionString'] = labels
    return submission_df

submission_df = generate_submission()
submission_df.to_csv('submission.csv')
!head submission.csv
submission_df